# Sample Database Code

Steal this code.

The TIdatabase module encapsulates all of the loading, storing and joining of the student, college and applications.

The module should be imported at the beginning of your code, whether it is a regular Python file or an iPython Notebook.

In [1]:
import TIdatabase as ti
import pandas as pd

## Generating Mock Data

In [2]:

students = ti.Student()
# populate students with random values
students.fillRandom(10)
students.df 

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp
0,U475Y42ZN4,0.854828,0.938297,0.829500,0.226419,0.602020,0.852946,3,4,2020,...,1,0,0,1,1,1,1,1,0,1
1,B2YQDGAH3N,0.188363,0.943160,0.807106,0.809517,0.212126,0.101765,2,3,2016,...,0,1,0,1,0,1,0,0,0,0
2,VNB2IAJ9QI,0.814618,0.879114,0.496170,0.471324,0.096854,0.770668,5,4,2019,...,0,1,0,1,1,0,0,0,1,0
3,PSLK5NPGK8,0.765792,0.667024,0.275380,0.304439,0.205852,0.739610,4,2,2013,...,1,0,1,1,0,1,1,0,1,0
4,6KVRKLC0IU,0.139412,0.169254,0.278542,0.194837,0.231334,0.536814,1,1,2013,...,1,0,0,1,0,0,1,0,0,1
5,HM5F7DTMCY,0.952215,0.195328,0.391965,0.388503,0.052430,0.968295,3,1,2010,...,1,1,1,1,1,1,1,1,1,0
6,0LGBTHBRW0,0.811959,0.258585,0.995958,0.072784,0.599447,0.565451,4,5,2020,...,0,1,0,1,0,1,1,0,0,0
7,5Q4WWOCYET,0.631654,0.707075,0.558682,0.230973,0.204389,0.288931,1,5,2016,...,1,0,0,0,0,1,0,1,0,1
8,ZF2UCP2EQY,0.879554,0.842051,0.970363,0.703324,0.142224,0.925094,3,5,2016,...,1,0,1,0,0,0,0,0,0,0
9,E8WZQLU67M,0.982381,0.798206,0.592648,0.250270,0.180391,0.710976,1,4,2010,...,1,0,0,0,0,0,1,0,0,0


The list of colleges is populated from a CSV stored in the same directory as this notebook. It can be edited using your favorite CSV editor, such as Excel. When you create a new instance of College, the values will be read in from the CSV

In [3]:
# populate with list of known colleges
colleges = ti.College()
colleges.df

,collegeID,name,acceptrate,size,public,finAidPct,instatePct
0,Princeton,Princeton,7.40%,5142,-1,0.00,0
1,Harvard,Harvard,6.00%,19929,-1,0.75,0
2,Yale,Yale,6.30%,12336,-1,0.00,0
3,Columbia,Columbia,7.00%,24221,-1,0.00,0
4,Stanford,Stanford,5.10%,16795,-1,0.00,0
5,UChicago,UChicago,8.80%,12558,-1,0.00,0
6,MIT,MIT,7.90%,11319,-1,0.00,0
7,Duke,Duke,11.40%,15856,-1,0.00,0
8,UPenn,UPenn,10.40%,21296,-1,0.00,0
9,CalTech,CalTech,8.80%,2209,-1,0.00,0


This is the table of application forms. "acceptStatus" is our predictor. "acceptProb" is our $\hat{Y}$ probability.

In [4]:
applForm = ti.ApplForm()
applForm.fillRandom(30)
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,PSLK5NPGK8,CarnegieMellon,0,0,0,0.046531
1,HM5F7DTMCY,UPenn,0,0,1,0.464016
2,PSLK5NPGK8,UPenn,0,0,1,0.111134
3,ZF2UCP2EQY,Stanford,0,0,1,0.739736
4,PSLK5NPGK8,NotreDame,1,1,0,0.840746
5,E8WZQLU67M,JohnsHopkins,1,0,0,0.154219
6,5Q4WWOCYET,UPenn,0,0,0,0.248705
7,B2YQDGAH3N,Vanderbilt,1,0,1,0.925449
8,U475Y42ZN4,Brown,0,1,0,0.980146
9,HM5F7DTMCY,UCB,0,1,1,0.211270


To combine the student and application forms tables, use Pandas merge capability, which will match rows by identical column names, which is studentID in this case:

In [5]:
applications = pd.merge(students.df,applForm.df)
applications

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,firstinfamily,alumni,sports,artist,workexp,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,U475Y42ZN4,0.854828,0.938297,0.829500,0.226419,0.602020,0.852946,3,4,2020,...,1,1,1,0,1,Brown,0,1,0,0.980146
1,B2YQDGAH3N,0.188363,0.943160,0.807106,0.809517,0.212126,0.101765,2,3,2016,...,1,0,0,0,0,Vanderbilt,1,0,1,0.925449
2,B2YQDGAH3N,0.188363,0.943160,0.807106,0.809517,0.212126,0.101765,2,3,2016,...,1,0,0,0,0,UChicago,1,0,1,0.219810
3,B2YQDGAH3N,0.188363,0.943160,0.807106,0.809517,0.212126,0.101765,2,3,2016,...,1,0,0,0,0,MIT,0,0,0,0.606733
4,VNB2IAJ9QI,0.814618,0.879114,0.496170,0.471324,0.096854,0.770668,5,4,2019,...,0,0,0,1,0,Brown,0,0,0,0.034319
5,VNB2IAJ9QI,0.814618,0.879114,0.496170,0.471324,0.096854,0.770668,5,4,2019,...,0,0,0,1,0,Northwestern,0,1,1,0.529486
6,PSLK5NPGK8,0.765792,0.667024,0.275380,0.304439,0.205852,0.739610,4,2,2013,...,1,1,0,1,0,CarnegieMellon,0,0,0,0.046531
7,PSLK5NPGK8,0.765792,0.667024,0.275380,0.304439,0.205852,0.739610,4,2,2013,...,1,1,0,1,0,UPenn,0,0,1,0.111134
8,PSLK5NPGK8,0.765792,0.667024,0.275380,0.304439,0.205852,0.739610,4,2,2013,...,1,1,0,1,0,NotreDame,1,1,0,0.840746
9,PSLK5NPGK8,0.765792,0.667024,0.275380,0.304439,0.205852,0.739610,4,2,2013,...,1,1,0,1,0,WashU,1,1,1,0.763763


Now the `applications` Pandas DataFrame is ready to use for either regression (by overwriting the acceptProb column) or building the public facing web site.

## Saving Scraped Data

### Part 1 - The Student Data

First, let's start fresh and delete the previously created objects. This is only necessary because this sample script is running within Jupyter where all variables are global to the page. In a separate Python file run from the command line, this step can be skipped. Creating a new instance would not work as behind the scenes, there is sharing of
DataFrames between objects.

In [6]:
if ('students' in locals()): 
    students.cleanup()
    del students
if ('applications' in locals()): del applications
if ('applForm' in locals()): del applForm


Let's create a new students instance. It will be an empty Pandas dataframe with the correct columns

In [7]:
students = ti.Student()
students.df

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp


Populate a dictionary with the values that the scraper has for a given row. Make sure the keys match up with the column names as only the matching columns will be saved. There is no need to add the studentID key. A unique value will be generated automatically.

In [8]:

# Example: international male who scored in 98th percentile in ACT/SAT, went to a public school and is applying for
# Class of 2020

newrow = {'admissionstest': 0.98,
         'GPA': 4.5,
         'female' : -1,
         'international': 1,
         'schooltype': -1,
         'intendedgradyear':2020}

students.insert(newrow)
students.df

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp
0,OO223JRZO9,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2020,...,-1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN


It is more efficient if multiple rows are added in one step. In this case, create a list of dictionaries and just
use the same method. Here, two new rows are added to the DataFrame in one step.

In [9]:
rows = []
a = {'schooltype': -1, 'admissionstest': 0.98, 'GPA': 4.5, 'female': 1, 'intendedgradyear': 2019, 'international': 0}
rows.append(a)
a = {'schooltype': 1, 'admissionstest': 0.65, 'GPA': 2.2, 'female': -1, 'intendedgradyear': 2019, 'international': 0}
rows.append(a)
students.insert(rows)
students.df

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp
0,OO223JRZO9,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2020,...,-1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
0,ARX8XCZ384,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2019,...,1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,UEP7SIT2I9,NaN,0.65,NaN,NaN,NaN,2.2,NaN,1,2019,...,-1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


Now we are ready to save. The data is saved in CSV format for ease of interpretability.

In [10]:
students.save("mydata.csv")

Let's delete all the data and check that we can read it back successfully.

In [11]:
if ('students' in locals()): 
    students.cleanup()
    del students
if ('applications' in locals()): del applications
if ('applForm' in locals()): del applForm



In [12]:
students = ti.Student()
students.read("mydata.csv")
students.df

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp
0,OO223JRZO9,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2020,...,-1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
0,ARX8XCZ384,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2019,...,1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,UEP7SIT2I9,NaN,0.65,NaN,NaN,NaN,2.2,NaN,1,2019,...,-1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


Et, voilà, the data is back.

### Part 2 - The Application Data

This is pretty much the same, **except** for two important differences:

* The studentID and collegeID must both be populated and exist in the respective DataFrames
* The columns are a little different. Normally acceptProb would not be populated from the scraper but could be used to store prediction runs. 

In [13]:
# we already wiped out applForm above
applForm = ti.ApplForm()
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb


In [14]:
studentID = students.df.iloc[1].studentID
collegeID = colleges.df.iloc[1].collegeID

In [15]:
print studentID, collegeID

ARX8XCZ384 Harvard


In [16]:
newrow = {'studentID': studentID,
         'collegeID': collegeID,
         'earlyAppl' : 0,
         'visited': 0,
         'acceptStatus': 0}

applForm.insert(newrow)
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,ARX8XCZ384,Harvard,0,0,0,NaN


Now inserting multiple rows

In [17]:
rows = []
studentID = students.df.iloc[2].studentID
collegeID = colleges.df.iloc[2].collegeID
newrow = {'studentID': studentID,
         'collegeID': collegeID,
         'earlyAppl' : 0,
         'visited': 0,
         'acceptStatus': 0}
rows.append(newrow)
# note: same student, new school to apply to
studentID = students.df.iloc[2].studentID
collegeID = colleges.df.iloc[3].collegeID
newrow = {'studentID': studentID,
         'collegeID': collegeID,
         'earlyAppl' : 1,
         'visited': 1,
         'acceptStatus': 1}
rows.append(newrow)
applForm.insert(rows)
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,ARX8XCZ384,Harvard,0,0,0,NaN
0,UEP7SIT2I9,Yale,0,0,0,NaN
1,UEP7SIT2I9,Columbia,1,1,1,NaN


Let's save it

In [18]:
applForm.save("applform1.csv")

Then delete the local variable

In [19]:
if ('applForm' in locals()): del applForm

Then read it back

In [20]:
applForm = ti.ApplForm()
applForm.read("applform1.csv")
applForm.df


,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,ARX8XCZ384,Harvard,0,0,0,NaN
0,UEP7SIT2I9,Yale,0,0,0,NaN
1,UEP7SIT2I9,Columbia,1,1,1,NaN


And now let's check that the merge still works

In [21]:
applications = pd.merge(students.df,applForm.df)
applications

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,firstinfamily,alumni,sports,artist,workexp,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,ARX8XCZ384,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2019,...,NaN,NaN,NaN,NaN,NaN,Harvard,0,0,0,NaN
1,UEP7SIT2I9,NaN,0.65,NaN,NaN,NaN,2.2,NaN,1,2019,...,NaN,NaN,NaN,NaN,NaN,Yale,0,0,0,NaN
2,UEP7SIT2I9,NaN,0.65,NaN,NaN,NaN,2.2,NaN,1,2019,...,NaN,NaN,NaN,NaN,NaN,Columbia,1,1,1,NaN


Ok, we are done for today.